In [ ]:
from httpx import get
from selectolax.parser import HTMLParser
import os
import logging

logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

def get_img_tags_for(term=None):
    if not term:
        raise Exception('No search term provided')
    url=f'https://unsplash.com/s/photos/{term}'
    response=get(url)
    if response.status_code!=200:
        raise Exception('Error getting response')
    tree=HTMLParser(response.text)
    imgs=tree.css('figure a img')
    return imgs

def img_filter_out(url: str,keywords: list) -> bool: 
    return not any(x in url for x in keywords)

def get_high_res_img_url(img_node):
    srcset=img_node.attrs['srcset']     
    srcset_list=srcset.split(', ')      
    url_res=[src.split(' ') for src in srcset_list if img_filter_out(src,['plus','premium','profile'])]     # filter out
    if not url_res:
        return None
    return url_res[0][0].split('?')[0]

img_nodes=get_img_tags_for('galaxy')
all_img_urls=[get_high_res_img_url(i) for i in img_nodes]
img_urls=[i for i in all_img_urls if i]

print(img_urls)

In [6]:
def save_imgs(img_urls,dest_dir='images',tag=''):

    for url in img_urls:
        response=get(url)
        file_name=url.split('/')[-1]
        logging.info(f'Downloading {url}...')
        if not os.path.exists(dest_dir):
            os.makedirs(dest_dir)

        with open(f'{dest_dir}/{tag}{file_name}.jpeg','wb') as f:
            f.write(response.content)
            logging.info(f'Saved {file_name}, with size {round(len(response.content)/1024/182,2)} MB')

In [ ]:
save_imgs(img_urls,'images','galaxy')